In [16]:
%%script echo skipping
from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/My\ Drive/Datasets/output.csv

file_path = '/content/drive/My Drive/Datasets/output.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'/content/drive/My Drive/Datasets/output.csv'


In [17]:
import pandas as pd
import numpy as np

In [18]:
file_path = '/home/remifigea/code/projet voice/divers/output.csv'
df = pd.read_csv(file_path)
df

,gloss,text
0,﻿MEMBERSHIP PARLIAMENT SEE MINUTE\n,﻿membership of parliament see minutes\n
1,APPROVAL MINUTE DESC-PREVIOUS SIT SEE MINUTE\n,approval of minutes of previous sitting see mi...
2,MEMBERSHIP PARLIAMENT SEE MINUTE\n,membership of parliament see minutes\n
3,VERIFICATION CREDENTIALS SEE MINUTE\n,verification of credentials see minutes\n
4,DOCUMENT RECEIVE SEE MINUTE\n,documents received see minutes\n
...,...,...
87705,NAME YOU WHAT(wh)\n,What is your name ?\n
87706,NAME YOU WHAT(wh)\n,What's your name?\n
87707,NAME YOU WHAT(wh)\n,What's your name ?\n
87708,EAT APPLE\n,I eat an apple\n


In [25]:
n_samples = 70000
maxlen= 8
max_features = 2000


In [20]:
from sklearn.utils import resample

data = resample(df, n_samples=n_samples)

In [21]:
! pip install contractions

In [22]:
import contractions
import re
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')


# deal with contractions, suppress 's possessif, suppress punctiation, numerical values, singularize
# keep only the maxlen-1 first gloss of each document to build dictionnary
def process_text(text, maxlen=maxlen):
    lemmatizer = WordNetLemmatizer()
    text = contractions.fix(text)
    text = re.sub(r"'s", "", text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    tokens = [t.lower() for t in tokens if t.isalpha()]
    tokens = [lemmatizer.lemmatize(token, pos='v') for token in tokens]
    tokens = [lemmatizer.lemmatize(token, pos='n') for token in tokens]
    tokens = tokens[:maxlen-1]
    return  tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
data['text_processed'] = data['text'].apply(process_text)
data['text_processed']

82372    [will, you, allow, your, fellow, member, to, a...
76860    [to, make, progress, we, will, propose, increa...
67578    [european, fund, can, and, should, at, most, p...
23107    [we, should, honour, those, who, fight, agains...
53576     [of, annual, gdp, and, this, money, be, not, be]
                               ...                        
4152     [link, to, this, discussion, be, that, relate,...
59387    [in, this, debate, we, have, hear, renew, plea...
52481                       [my, answer, go, without, say]
41971      [a, number, of, key, issue, have, not, yet, be]
56668    [though, i, be, ashamed, i, have, to, agree, w...
Name: text_processed, Length: 87710, dtype: object

In [26]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [28]:
#keep only the maxlen-1 first gloss of each document to build dictionnary
# remove punctuation excpet (,),-, remove numerical values, add START and END, remove ''
def process_gloss(document, maxlen=maxlen):
    punctuation_pattern = r"[^\w\s()-]|(\d+)"
    gloss_list = document.split()
    gloss_list = [re.sub(punctuation_pattern, '', gloss) for gloss in gloss_list]
    gloss_list = [gloss for gloss in gloss_list if gloss!='' ]
    gloss_list.append('END')
    gloss_list.insert(0, 'START')
    gloss_list = gloss_list[:maxlen-1]
    return  gloss_list

In [29]:
#preprocess gloss
data['gloss_processed'] = data['gloss'].apply(process_gloss)
data['gloss_processed']

82372    [START, WILL, X-YOU, ALLOW, X-YOU, FELLOW, MEM...
76860    [START, TO, MAKE, PROGRESS, X-WE, WILL, PROPOS...
67578    [START, EUROPEAN, FUND, CAN, AND, SHOULD, AT, ...
23107    [START, X-WE, SHOULD, HONOR, THOSE, WHO, FIGHT...
53576    [START, DESC-ANNUAL, GDP, AND, THIS, MONEY, BE...
                               ...                        
4152     [START, LINK, TO, THIS, DISCUSSION, BE, THAT, ...
59387    [START, IN, THIS, DEBATE, X-WE, HAVE, HEAR, RE...
52481         [START, X-MY, ANSWER, GO, WITHOUT, SAY, END]
41971    [START, NUMBER, KEY, ISSUE, HAVE, DESC-NOT, DE...
56668    [START, THOUGH, X-I, BE, DESC-ASHAMED, X-I, HA...
Name: gloss_processed, Length: 87710, dtype: object

In [30]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(data, test_size=0.2, random_state=1)

In [31]:
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

# prepare X_train, X_test

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_input = CountVectorizer(max_features=max_features, analyzer=lambda x: x)
vectorizer_input.fit([['pommes', 'pomme']])
vocab_tokens_input = vectorizer_input.get_feature_names_out()
len(vocab_tokens_input)
dict_vocab_input_to_idx = vectorizer_input.vocabulary_


In [33]:
#Build a dictionnary for input_word
#create a BOW
from sklearn.feature_extraction.text import CountVectorizer

# We create the output BOW, we can even reject directly the stop words and the punctuation, how magical?
vectorizer_input = CountVectorizer(max_features=max_features, analyzer=lambda x: x)
vectorizer_input.fit(df_train['text_processed'])

vocab_tokens_input = vectorizer_input.get_feature_names_out()
len(vocab_tokens_input)

3000

In [36]:
#%%script echo skipping
BOW_input = vectorizer_input.transform(df_train['text_processed']).toarray()
dict_vocab_input_to_idx = vectorizer_input.vocabulary_

# return the number of occurences of a word in df_train
def get_token_occurences(token):
    dict_vocab_input_to_idx = vectorizer_input.vocabulary_
    index_token = dict_vocab_input_to_idx.get(token)
    if index_token is not None:
        nombre_occurrences = BOW_input[:, index_token].sum()
        return nombre_occurrences
    else:
        return 0

In [37]:
get_token_occurences('path')

50

In [38]:
dict_vocab_input_to_idx = {token:i+2 for i,token in enumerate(vocab_tokens_input) }
dict_vocab_input_to_idx['END'] = 0
dict_vocab_input_to_idx['START'] = 1
dict_vocab_input_to_idx

{'a': 2,
 'abandon': 3,
 'ability': 4,
 'able': 5,
 'abolish': 6,
 'abolition': 7,
 'abortion': 8,
 'about': 9,
 'above': 10,
 'absence': 11,
 'absolute': 12,
 'absolutely': 13,
 'abstain': 14,
 'abstention': 15,
 'absurd': 16,
 'abuse': 17,
 'accelerate': 18,
 'accept': 19,
 'acceptable': 20,
 'access': 21,
 'accessible': 22,
 'accession': 23,
 'accident': 24,
 'accompany': 25,
 'accord': 26,
 'accordance': 27,
 'accordingly': 28,
 'account': 29,
 'accountability': 30,
 'accountable': 31,
 'accusation': 32,
 'accuse': 33,
 'achieve': 34,
 'achievement': 35,
 'acknowledge': 36,
 'acp': 37,
 'across': 38,
 'act': 39,
 'acta': 40,
 'action': 41,
 'active': 42,
 'actively': 43,
 'activity': 44,
 'actor': 45,
 'actual': 46,
 'actually': 47,
 'ad': 48,
 'adapt': 49,
 'adaptation': 50,
 'add': 51,
 'addiction': 52,
 'addition': 53,
 'additional': 54,
 'address': 55,
 'adequate': 56,
 'adhere': 57,
 'adjourn': 58,
 'adjournment': 59,
 'adjust': 60,
 'adjustment': 61,
 'administration': 62,
 '

In [39]:
dict_idx_to_vocab_input = {i+2:token for i, token in enumerate(vocab_tokens_input) }
dict_idx_to_vocab_input[0] = 'END'
dict_idx_to_vocab_input[1] = 'START'
dict_idx_to_vocab_input

{2: 'a',
 3: 'abandon',
 4: 'ability',
 5: 'able',
 6: 'abolish',
 7: 'abolition',
 8: 'abortion',
 9: 'about',
 10: 'above',
 11: 'absence',
 12: 'absolute',
 13: 'absolutely',
 14: 'abstain',
 15: 'abstention',
 16: 'absurd',
 17: 'abuse',
 18: 'accelerate',
 19: 'accept',
 20: 'acceptable',
 21: 'access',
 22: 'accessible',
 23: 'accession',
 24: 'accident',
 25: 'accompany',
 26: 'accord',
 27: 'accordance',
 28: 'accordingly',
 29: 'account',
 30: 'accountability',
 31: 'accountable',
 32: 'accusation',
 33: 'accuse',
 34: 'achieve',
 35: 'achievement',
 36: 'acknowledge',
 37: 'acp',
 38: 'across',
 39: 'act',
 40: 'acta',
 41: 'action',
 42: 'active',
 43: 'actively',
 44: 'activity',
 45: 'actor',
 46: 'actual',
 47: 'actually',
 48: 'ad',
 49: 'adapt',
 50: 'adaptation',
 51: 'add',
 52: 'addiction',
 53: 'addition',
 54: 'additional',
 55: 'address',
 56: 'adequate',
 57: 'adhere',
 58: 'adjourn',
 59: 'adjournment',
 60: 'adjust',
 61: 'adjustment',
 62: 'administration',
 6

In [40]:
def convert_into_idx(document):
    doc_to_idx = [dict_vocab_input_to_idx.get(token) for token in document if token in dict_vocab_input_to_idx.keys()]
    #for token START
    doc_to_idx.insert(0,1)
    return doc_to_idx

In [41]:
df_train['text_to_idx'] = df_train['text_processed'].apply(convert_into_idx)
df_train['text_to_idx']

0        [1, 1721, 1505, 140, 1393, 1745, 268, 1828, 1849]
1        [1, 2337, 2282, 1116, 429, 2311, 1253, 1805, 2...
2        [1, 1849, 2687, 2897, 1577, 2728, 2917, 1253, ...
3          [1, 2687, 2451, 268, 2626, 213, 140, 2295, 213]
4        [1, 2687, 1836, 2702, 2453, 268, 1336, 366, 2687]
                               ...                        
70163                    [1, 2917, 1504, 2872, 2929, 2932]
70164    [1, 1322, 2694, 1061, 2686, 2917, 2953, 2802, ...
70165              [1, 2917, 1745, 1190, 1836, 2702, 2642]
70166    [1, 2687, 961, 2825, 268, 1805, 1151, 2598, 2674]
70167          [1, 1322, 1253, 1133, 54, 1851, 2726, 1612]
Name: text_to_idx, Length: 70168, dtype: object

In [42]:
df_train['lenght'] = df_train['text_to_idx'].apply(lambda x: len(x))
df_train['lenght'].describe()

count    70168.000000
mean         8.867618
std          1.809666
min          1.000000
25%          8.000000
50%         10.000000
75%         10.000000
max         10.000000
Name: lenght, dtype: float64

In [43]:
#perform a padding for mac sequence=15
X_train = np.asarray(df_train['text_to_idx'])

In [44]:
df_test['text_to_idx'] = df_test['text_processed'].apply(convert_into_idx)
df_test['lenght'] = df_test['text_to_idx'].apply(lambda x: len(x))
df_test['lenght'].describe()

count    17542.000000
mean         8.842207
std          1.825043
min          1.000000
25%          8.000000
50%         10.000000
75%         10.000000
max         10.000000
Name: lenght, dtype: float64

In [45]:
#perform a padding for mac sequence=15
X_test = np.asarray(df_test['text_to_idx'])

In [46]:
from tensorflow.keras.preprocessing import sequence

X_train_seq = sequence.pad_sequences(X_train,
                                 value=0,
                                 padding='post', # to add zeros at the end
                                 truncating='post', # to cut the end of long sequences
                                 maxlen=maxlen) # the length we want

X_test_seq = sequence.pad_sequences(X_test,
                                 value=0,
                                 padding='post', # to add zeros at the end
                                 truncating='post', # to cut the end of long sequences
                                 maxlen=maxlen) # the length we want

X_train_seq, X_test_seq

(array([[   1, 1721, 1505, ..., 1828, 1849,    0],
        [   1, 2337, 2282, ..., 1805, 2989,  268],
        [   1, 1849, 2687, ..., 1253, 2687, 2897],
        ...,
        [   1, 2917, 1745, ...,    0,    0,    0],
        [   1, 2687,  961, ..., 2598, 2674,    0],
        [   1, 1322, 1253, ..., 1612,    0,    0]], dtype=int32),
 array([[   1, 2687, 2565, ...,  140, 2687,  350],
        [   1, 1459, 2615, ..., 1365, 2142, 1525],
        [   1, 2687,  679, ..., 1365, 2687, 1240],
        ...,
        [   1, 2702, 1253, ..., 2687,  660, 1836],
        [   1, 1884,  442, ..., 1854, 2734,    0],
        [   1,    2, 1253, ..., 1983,    2,    0]], dtype=int32))

# prepare Y_train, Y_test

In [47]:
# We create the output BOW, we can even reject directly the stop words and the punctuation, how magical?
vectorizer_output = CountVectorizer(max_features=max_features, analyzer=lambda x: x)
vectorizer_output.fit(df_train['gloss_processed'])

gloss_output = vectorizer_output.get_feature_names_out()
len(gloss_output)

3000

In [48]:
%%script echo skipping
BOW_output = vectorizer_output.transform(df_train['gloss_processed']).toarray()
dict_vocab_output_to_idx = vectorizer_output.vocabulary_

# return the number of occurences of a gloss in df_train
def get_gloss_occurences(gloss):
    dict_vocab_output_to_idx = vectorizer_output.vocabulary_
    index_gloss = dict_vocab_output_to_idx.get(gloss)
    if index_gloss is not None:
        nombre_occurrences = BOW_output[:, index_gloss].sum()
        return nombre_occurrences
    else:
        return 0
        

In [49]:
vectorizer_output.transform(df_train['gloss_processed']).toarray()
dict_vocab_output_to_idx = vectorizer_output.vocabulary_


50

In [50]:
gloss_output

array(['ABANDON', 'ABILITY', 'ABOLISH', ..., 'ZERO', 'ZIMBABWE', 'ZONE'],
      dtype=object)

In [51]:
gloss_list = gloss_output.tolist()
gloss_list

['ABANDON',
 'ABILITY',
 'ABOLISH',
 'ABOLITION',
 'ABORTION',
 'ABOUT',
 'ABOVE',
 'ABSENCE',
 'ABSTAIN',
 'ABSTENTION',
 'ABUSE',
 'ACCELERATE',
 'ACCEPT',
 'ACCESS',
 'ACCESSION',
 'ACCIDENT',
 'ACCOMPANY',
 'ACCORD',
 'ACCORDANCE',
 'ACCOUNT',
 'ACCOUNTABILITY',
 'ACCUSATION',
 'ACCUSE',
 'ACHIEVE',
 'ACHIEVEMENT',
 'ACKNOWLEDGE',
 'ACP',
 'ACQUI',
 'ACROSS',
 'ACT',
 'ACTA',
 'ACTION',
 'ACTIVIST',
 'ACTIVITY',
 'ACTOR',
 'ADAPT',
 'ADAPTATION',
 'ADD',
 'ADDITION',
 'ADDRESS',
 'ADHERE',
 'ADJOURN',
 'ADJOURNMENT',
 'ADJUST',
 'ADJUSTMENT',
 'ADMINISTRATION',
 'ADMIT',
 'ADOPT',
 'ADOPTION',
 'ADVANCE',
 'ADVANTAGE',
 'ADVICE',
 'ADVOCATE',
 'AFFAIR',
 'AFFECT',
 'AFFIRM',
 'AFFORD',
 'AFGHAN',
 'AFGHANISTAN',
 'AFRICA',
 'AFRICAN',
 'AFTER',
 'AFTERNOON',
 'AGAINST',
 'AGE',
 'AGEING',
 'AGENCY',
 'AGENDA',
 'AGREE',
 'AGREEMENT',
 'AGRICULTURE',
 'AID',
 'AIM',
 'AIR',
 'AIRCRAFT',
 'AIRLINE',
 'AIRPORT',
 'AIRSPACE',
 'AL',
 'ALARM',
 'ALBANIUM',
 'ALCOHOL',
 'ALDE',
 'ALL',
 

In [52]:
len(gloss_list)

3000

In [53]:
def BOW_custom(document, gloss_list=gloss_list, maxlen=maxlen, max_features=max_features):

    y = np.zeros((maxlen, max_features))
    i = 0
    for gloss in document:
        # keep only maxlen gloss
        if i<maxlen:
            if gloss in gloss_list:
                idx_gloss = gloss_list.index(gloss)
                y[i, idx_gloss] = 1
                i=i+1
    # fill in with 'END' GLOSS to reach maxlen gloss
    idx_END_gloss = gloss_list.index('END')
    while i<maxlen:
        y[i, idx_END_gloss] = 1
        i = i + 1

    return y

In [54]:
y=BOW_custom(df_train['gloss_processed'][4])

In [55]:
y.shape

(10, 3000)

In [56]:
df_train['gloss_processed'][4]

['START',
 'ABSURDITY',
 'THIS',
 'SITUATION',
 'BE',
 'ILLUSTRATE',
 'BY',
 'EXAMPLE',
 'THAT']

In [57]:
#check
for i in range(maxlen):
    print(gloss_output[y[i]==1])

['START']
['THIS']
['SITUATION']
['BE']
['ILLUSTRATE']
['BY']
['EXAMPLE']
['THAT']
['END']
['END']


In [54]:
#pd.set_option('display.max_colwidth', None)

In [55]:
#df_train

In [58]:
y_train = np.zeros((df_train.shape[0], maxlen, max_features), dtype=np.uint8)

In [59]:
y_train.shape

(70168, 10, 3000)

In [60]:
from tqdm import tqdm

for idx in tqdm(range(df_train.shape[0])):
    y_train[idx] = BOW_custom(df_train['gloss_processed'][idx])

100%|██████████| 70168/70168 [01:18<00:00, 899.08it/s] 


In [62]:
y_train.shape

(70168, 10, 3000)

In [63]:
y_test = np.zeros((df_test.shape[0], maxlen, max_features), dtype=np.uint8)

In [64]:
for idx in tqdm(range(df_test.shape[0])):
    y_test[idx] = BOW_custom(df_test['gloss_processed'][idx])

100%|██████████| 17542/17542 [00:18<00:00, 970.81it/s]


In [65]:
# return the number of occurences of a gloss in df_train
def get_gloss_occurences_2(gloss):
    gloss_idx = gloss_list.index(gloss)
    gloss_count = 0
    if gloss_idx is not None:
        for seq in range(y_train.shape[0]):
            gloss_count = y_train[seq][:, gloss_idx].sum() +  gloss_count
    return gloss_count

In [66]:
get_gloss_occurences_2('PATH')

50.0

In [67]:
import tensorflow as tf
from nltk.translate.bleu_score import sentence_bleu

In [68]:
def bleu_score(y_true, y_pred):
    y_true = tf.argmax(y_true, axis=-1)
    y_pred = tf.argmax(y_pred, axis=-1)

    score = tf.py_function(func=calculate_bleu, inp=[y_true, y_pred], Tout=tf.float32)
    return score

def calculate_bleu(y_true, y_pred):
    score = 0
    y_true = y_true.numpy()
    y_pred = y_pred.numpy()

    for true_seq, pred_seq in zip(y_true, y_pred):
        true_seq = [str(token) for token in true_seq if token != 0]
        pred_seq = [str(token) for token in pred_seq if token != 0]
        score += sentence_bleu([true_seq], pred_seq)

    return score / len(y_true)

In [69]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, TimeDistributed, Bidirectional,  Dropout
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime

current_datetime = datetime.now().strftime("%d_%H%M")
parameters = 'p=0.1'

# Define the callbacks
callbacks = [TensorBoard(log_dir=f'./Graph/{current_datetime}-{parameters}', histogram_freq=0, write_graph=True, write_images=True)]


def my_RNN(max_features=max_features, maxlen=maxlen, dropout= 0.1):

    model = Sequential()
    # The input_dim is the number of different words we have in our corpus: here 5000
    # The input_length is the length of our sequences: here 15 thanks to padding
    model.add(Embedding(input_dim=max_features+2, output_dim=32, input_length=maxlen))

    # We add two layers of RNN
    model.add(Bidirectional(SimpleRNN(units=32, return_sequences=True)))
    model.add(Dropout(dropout))
    model.add(Bidirectional(SimpleRNN(units=32, return_sequences=True)))
    model.add(Dropout(dropout))

    # Finally we add a sigmoid
    model.add(TimeDistributed(Dense(max_features, activation='softmax')))

    return model

In [70]:
model = my_RNN()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', bleu_score])

In [71]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 10, 32)            96064     
                                                                 
 bidirectional_2 (Bidirecti  (None, 10, 64)            4160      
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 10, 64)            0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 10, 64)            6208      
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 10, 64)            0         
                                                                 
 time_distributed_1 (TimeDi  (None, 10, 3000)         

In [ ]:
model.fit(x=X_train_seq, y=y_train, #validation_data=(X_test_seq, y_test),\
          epochs=50, batch_size=128, callbacks=callbacks)

Epoch 1/60


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

295/549 [===============>..............] - ETA: 57s - loss: 4.8930 - accuracy: 0.3207 - bleu_score: 0.0639

In [ ]:
! tensorboard --logdir=Graph

In [ ]:
model.save_weights(f'./poids{current_datetime}-{parameters}.h5')

In [ ]:
%%script echo skipping
!cp '/content/poids{current_datetime}-{parameters}.h5' /content/drive/MyDrive/

In [ ]:
def retrieve_gloss_from_prediction(prediction, gloss_list=gloss_list):
    gloss_conversion = []
    for row_idx in range(maxlen):
        gloss_predicted = gloss_list[np.argmax((prediction[0][row_idx]))]
        gloss_conversion.append(gloss_predicted)

    return gloss_conversion

In [ ]:
import random as rd

num = rd.randint(0, len(X_test_seq))

document_to_tranlate = X_test_seq[num:num+1]
prediction = model.predict(document_to_tranlate)

#predicted gloss
print(retrieve_gloss_from_prediction(prediction))
#real gloss
for i in range(maxlen):
    print(gloss_output[y_test[num][i]==1], gloss_output[y_test[num][i]==1] in gloss_list)
# initial text
print([dict_idx_to_vocab_input.get(idx) for idx in document_to_tranlate[0]])


In [ ]:
sentence = 'i will have a break soon'
document = sentence.split()
X = np.asarray(convert_into_idx(document)).reshape(1,-1)
X

In [ ]:
sentence = 'i played football'
document = sentence.split()
X = np.asarray(convert_into_idx(document)).reshape(1,-1)
X_seq = sequence.pad_sequences(X,
                                 value=0,
                                 padding='post', # to add zeros at the end
                                 truncating='post', # to cut the end of long sequences
                                 maxlen=maxlen)
prediction = model.predict(X_seq)
print(retrieve_gloss_from_prediction(prediction))
for word in document:
    print(word, word in gloss_list)

In [ ]:
for i in range(maxlen):
    print(np.sort(prediction[0][i])[-5:])